## Q-Learning - Robots en Warehouse

https://www.analyticsvidhya.com/blog/2021/04/q-learning-algorithm-with-step-by-step-implementation-using-python/

In [9]:
# Importar librerias
import numpy as np

In [10]:
# Definir el Environment
environment_rows = 11
environment_columns = 11
# Crear array 3D para almacenar los Q-values actuales para cada par estado/acción: Q(s, a) 
# El array tiene 11 filas y 11 columnas (environment), y una tercera dimensión "action".
# La dimensión "action" tiene 4 capas para hacer un seguimiento de los Q-values para cada posible acción
# en cada estado (hay 4 posibles acciones). 
# Inicializar el valor de cada par (estado,acción) a cero.
q_values = np.zeros((environment_rows, environment_columns, 4))

![Warehouse_Map_con_Rewards](warehouse-map-rewards.png)

In [11]:
# Definir acciones
# Códigos numéricos de acciones: 0 = up, 1 = right, 2 = down, 3 = left
actions = ['up', 'right', 'down', 'left']

In [12]:
# Crear array 2D para almacenar los rewards para cada estado. 
# El array tiene 11 filas por 11 columnas (environment), y cada valor se inicializa en -100.

rewards = np.full((environment_rows, environment_columns), -100.)
rewards[0, 5] = 100. # Cargar reward para el recuadro verde (la meta-objetivo) en 100

# Definir las ubicaciones de los pasillos (recuadros blancos) para las filas 1 a 9
aisles = {} # almacenar ubicaciones en un diccionario
aisles[1] = [i for i in range(1, 10)]
aisles[2] = [1, 7, 9]
aisles[3] = [i for i in range(1, 8)]
aisles[3].append(9)
aisles[4] = [3, 7]
aisles[5] = [i for i in range(11)]
aisles[6] = [5]
aisles[7] = [i for i in range(1, 10)]
aisles[8] = [3, 7]
aisles[9] = [i for i in range(11)]
# Cargar valores de reward para las posiciones de los pasillos (recuadros blancos)
for row_index in range(1, 10):
  for column_index in aisles[row_index]:
    rewards[row_index, column_index] = -1.
# Imprimir matriz de rewards
for row in rewards:
  print(row)

[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]


In [13]:
# Definir algunas funciones auxiliares
# Definir función para saber si la ubicación es un estado terminal
def is_terminal_state(current_row_index, current_column_index):
  # si el reward para esta ubicación = -1, no es estado terminal (ej.: es un recuadro blanco)
  if rewards[current_row_index, current_column_index] == -1.:
    return False
  else:
    return True
# Definir función para seleccionar ubicación de inicio (aleatoria y no terminal)
def get_starting_location():
  # Obtener índice de fila y columna aleatoriamente
  current_row_index = np.random.randint(environment_rows)
  current_column_index = np.random.randint(environment_columns)
  # Continuar seleccionado aleatoriamente filas/columnas hasta identificar ubicación no terminal
  while is_terminal_state(current_row_index, current_column_index):
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_columns)
  return current_row_index, current_column_index
# Definir algoritmo epsilon greedy para elegir próxima acción
def get_next_action(current_row_index, current_column_index, epsilon):
  # si valor aleatorio (entre 0 y 1) menor a epsilon, entonces elegir el mejor valor de Q-table. 
  if np.random.random() < epsilon:
    return np.argmax(q_values[current_row_index, current_column_index])
  else: # seleccionar acción aleatoria
    return np.random.randint(4)
# Definir función que obtiene próxima ubicación según la acción seleccionada
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index
  if actions[action_index] == 'up' and current_row_index > 0:
    new_row_index -= 1
  elif actions[action_index] == 'right' and current_column_index < environment_columns - 1:
    new_column_index += 1
  elif actions[action_index] == 'down' and current_row_index < environment_rows - 1:
    new_row_index += 1
  elif actions[action_index] == 'left' and current_column_index > 0:
    new_column_index -= 1
  return new_row_index, new_column_index
# Definir función para obtener el recorrido más corto entre cualquier ubicación dentro del warehouse 
def get_shortest_path(start_row_index, start_column_index):
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else:
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    # Continuar moviéndose hasta alcanzar meta (ej.: recuadro verde)
    while not is_terminal_state(current_row_index, current_column_index):
      # Obtener mejor acción a tomar
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      # Moverse a la próxima ubicación, y agregar ubicación nueva a la lista
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [14]:
# Entrenar el agente usando el algoritmo Q-Learning

# Definir parámetros de entrenamiento

epsilon = 0.9 # Algoritmo epsilon greedy para exploración/explotación
discount_factor = 0.9 # discount factor para rewards futuros
learning_rate = 0.9 # learning rate

# Ejecutar 1000 episodios de entrenamiento

for episode in range(1000):
  # Obtener posición inicio para este episodio
  row_index, column_index = get_starting_location()
  # Continuar tomando acciones (moviéndose) hasta alcanzar un estado terminal (Fin episodio)
  # (ej.: hasta alcanzar recuadro verde o chocar con ubicación de almacenamiento)
  while not is_terminal_state(row_index, column_index):
    # Elegir próxima acción a tomar
    action_index = get_next_action(row_index, column_index, epsilon)
    # Ejecutar acción y pasar al próximo estado
    old_row_index, old_column_index = row_index, column_index # Almacenar índices viejos de fila y columna
    row_index, column_index = get_next_location(row_index, column_index, action_index)
    # Obtener el reward por el nuevo estado, y calcular diferencia temporal (TD)
    reward = rewards[row_index, column_index]
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value
    # Actualizar Q-value para el par anterior estado/acción
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value
print('Entrenamiento completado!')

Entrenamiento completado!


In [15]:
# Después de haber entrenado, visualizar el recorrido más corto entre diferentes ubicaciones

print(get_shortest_path(3, 9)) #starting at row 3, column 9
print(get_shortest_path(5, 0)) #starting at row 5, column 0
print(get_shortest_path(9, 5)) #starting at row 9, column 5

[[3, 9], [2, 9], [1, 9], [1, 8], [1, 7], [1, 6], [1, 5], [0, 5]]
[[5, 0], [5, 1], [5, 2], [5, 3], [4, 3], [3, 3], [3, 2], [3, 1], [2, 1], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [0, 5]]
[[9, 5], [9, 6], [9, 7], [8, 7], [7, 7], [7, 6], [7, 5], [6, 5], [5, 5], [5, 6], [5, 7], [4, 7], [3, 7], [2, 7], [1, 7], [1, 6], [1, 5], [0, 5]]


In [16]:
print(q_values)

[[[   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]
  [   0.            0.            0.            0.        ]]

 [[   0.            0.            0.            0.        ]
  [ -99.999        62.171        47.9403813   -99.9       ]
  [ -99.9          70.19        -99.99         33.84463684]
  [ -99.           79.1         -99.99         62.10711981]
  [ -99.           89.          -99.99         70.182981  ]
  [ 100.           79.1        -100.  